# Testing

## Preparation

In [2]:
import cProfile as cp
import pstats
from pstats import SortKey
from typing import Sequence, Dict

In [3]:
def clean_stats(stats: pstats.Stats):
    '''
    Reduces an instance pstats.Stats to only function name and values for "ncalls", "cumtime", "percall", and "tottime".
    ncalls:   How often a function was called
    cumtime:  How much time was spent in the function (including calls by this function)
    percall:  How much time was spent per call (without subcalls)
    tottime:  How much time was spent in the function (without subcalls)
    '''
    cleaned = {}
    profiles = stats.get_stats_profile().func_profiles
    for func_name, func_profile in profiles.items():
        cleaned[func_name] = {'ncalls': func_profile.ncalls,
                              'cumtime': func_profile.cumtime,
                              'percall': func_profile.percall_tottime, 
                              'tottime': func_profile.tottime}
    return cleaned

def get_stats(module_name: str, num: int):
    '''
    Function that profiles the mysqrt function of a provided module following the following documentation
    https://docs.python.org/3/library/profile.html
    
    :param module_name:  Name of the module to be evaluated (Use the import name)
    :param num:          Number to evaluate the model on
    '''
    
    with cp.Profile() as profile:
        eval(module_name).mysqrt(num)
        profile.dump_stats(module_name)
        p = pstats.Stats(module_name)
        p.strip_dirs().sort_stats(SortKey.CUMULATIVE)
    return clean_stats(p)

In [4]:
def evaluate_module(module_name: str, numbers: Sequence[int]):
    '''
    Evaluate one module over all numbers 
    '''
    return [get_stats(module_name, num) for num in numbers]

def extract_summary(results: Sequence[Dict], func_name: str, attribute='tottime'):
    '''
    For each result in the results list, get the requested attribute for the requested function (fnc_name)
    '''
    return [result[func_name][attribute] for result in results]

## Implementation 1

In [5]:
import versions.schmidt_hw2 as v1

numbers = [1234, 12345]
results = evaluate_module("v1", numbers)

In [6]:
mysqrt = extract_summary(results, 'mysqrt', attribute='cumtime')
lowupper = extract_summary(results, 'getLowUpper', attribute='cumtime')
isperfect = extract_summary(results, 'isperfect', attribute='cumtime')

print(f'mysqrt: {mysqrt}')
print(f'lowupper: {lowupper}')
print(f'isperfect: {isperfect}')

mysqrt: [0.015, 0.472]
lowupper: [0.015, 0.47]
isperfect: [0.015, 0.471]


As seen by the values for cumulative time, the most time is spent in `isperfect`. We can also see that it is called a low by `getLowUpper`:

In [7]:
print(extract_summary(results, 'isperfect', attribute='ncalls'))

['72', '224']


In order to address this, the looping in getLowUpper will be replaced using vectorization

## Adaption 1: Vectorize getLowUpper

In [8]:
import numpy as np

In order to vectorize this process we need to create an array. To not create an unnecessary large array, we need to find the maximum distance from an integer to the next perfect square. This can be found as follows:

1. Let `x` be an integer, then its square is given by `x^2`
2. The distance to the adjacent squared integers is given by `x^2 - (x-1)^2 = 2x - 1` and `(x+1)^2 - x^2 = 2x + 1` respectively 
3. Now let `n = x^2`. Then, the distances to the next perfect square from `n` relative to the value of `n` become `2/sqrt(n) - 1/z` and `2/sqrt(n) + 1/z`
4. As the value for `n` increases, the relative distance to the next perfect square decreases.

**This allows us to find the size of the arrays in which to look for adjacent perfect squares:**

1. As the relative distances decrease, a fixed percentage that works for `n` will also work for all integers larger than `n`
2. As the absolute distances increase, a fixed absolute value that works for `n` will also work for all integers smaller than `n`.

Below, I will start with `n=2500`, for which the relative distance is `4.1%` and the absolute distance is `101`. 

In [9]:
def create_ranges(n: int):
    distance = max(101, int(n*0.041))
    low_end = n - distance
    high_end = n + distance
                   
    return np.arange(low_end, n, 1), np.arange(n, high_end + 1, 1)

In [10]:
n = 42
lower_arr, upper_arr = create_ranges(n)

The next step is to apply `isperfect` to both arrays and return the two closest perfect squares to the integer `n` for which to estimate the square root.
This can be achieved by filtering both arrays for `True` values and returning the largest value in the first array and the smallest value in the second array. 

The following implementation using the boolean array returned by the vectorization as a mask on the second array
See also: https://stackoverflow.com/questions/19984102/select-elements-of-numpy-array-via-boolean-mask-array

In [11]:
def reduce_array(arr: np.array):
    '''
    Receives an array of integer candidates and return only the elements for which "isperfect" returns true
    '''
    mask_arr, int_arr = np.vectorize(v1.isperfect)(arr)
    return int_arr[mask_arr]

In [12]:
print(reduce_array(lower_arr))
print(reduce_array(upper_arr))

print(np.max(reduce_array(lower_arr)))
print(np.min(reduce_array(upper_arr)))

[0 1 2 3 4 5 6]
[ 7  8  9 10 11]
6
7


Now, combine these function into a new `getLowUpper` helper:

In [13]:
def getLowUpper_v2(n: int):
    """
        This function is the second helper. It takes an integer n and returns the lower and upper perfect square root to n.
        We will use two "while" loops here, but we could have used "for" loops or whatever.
        The first that will catch the first perfect square root is less than the square root of n.
        The second one will catch the first square root greater than the square root of n.

        INPUT: n as an integer.
        OUTPUT: a tuple (minsqrt:int, maxsqrt:int)

        Examples:
        getLowUpper(3) = (1,2)
        getLowUpper(15) = (3,4)
    """
    lower_arr, upper_arr = create_ranges(n)
    minsqrt = np.max(reduce_array(lower_arr))
    maxsqrt = np.min(reduce_array(upper_arr))

    return minsqrt, maxsqrt

### Compare v1 and v2

Unfortunately, this implementation is not faster despite vectorization:

In [14]:
cp.run("v1.getLowUpper(12345)")

         227 function calls in 0.481 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.481    0.481 <string>:1(<module>)
        1    0.000    0.000    0.481    0.481 schmidt_hw2.py:31(getLowUpper)
      223    0.480    0.002    0.480    0.002 schmidt_hw2.py:6(isperfect)
        1    0.000    0.000    0.481    0.481 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [15]:
cp.run("getLowUpper_v2(12345)")

         1093 function calls in 2.184 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 3018333493.py:1(create_ranges)
        1    0.000    0.000    2.184    2.184 3250889149.py:1(getLowUpper_v2)
        2    0.000    0.000    2.184    1.092 71781565.py:1(reduce_array)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amax)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amin)
        1    0.000    0.000    2.184    2.184 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2672(_amax_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2677(amax)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2797(_amin_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2802(amin)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:69(_wrap

One possible explanation for the worse performance is that the array is too large. Hence, instead of ensuring that the array is guaranteed to include an adjacent perfect square, I will now implement smaller arrays that are tested iteratively with arrays of varying sizes.

In [39]:
def getLowUpper_v3(n: int, array_size=20):
    """
        This function is the second helper. It takes an integer n and returns the lower and upper perfect square root to n.
        We will use two "while" loops here, but we could have used "for" loops or whatever.
        The first that will catch the first perfect square root is less than the square root of n.
        The second one will catch the first square root greater than the square root of n.

        INPUT: n as an integer.
        OUTPUT: a tuple (minsqrt:int, maxsqrt:int)

        Examples:
        getLowUpper(3) = (1,2)
        getLowUpper(15) = (3,4)
    """
    lower_found = False
    lower_start = n
    while not lower_found:
        lower_arr = np.arange(lower_start-array_size, lower_start, 1)
        mask_arr, int_arr = np.vectorize(v1.isperfect)(lower_arr)
        lower_found = True in mask_arr
        lower_start -= array_size
        
    minsqrt = np.max(int_arr[mask_arr])
    
    upper_found = False
    upper_start = n
    while not upper_found:
        upper_arr = np.arange(upper_start, upper_start + array_size, 1)
        mask_arr, int_arr = np.vectorize(v1.isperfect)(upper_arr)
        upper_found = True in mask_arr
        upper_start += array_size

    maxsqrt = np.min(int_arr[mask_arr])  

    return minsqrt, maxsqrt

Trying different array sizes shows that this approach is faster than v2 but still slower than the original code (despite vectorization)

In [34]:
cp.run("getLowUpper_v3(12345, 5)")

         1505 function calls in 0.591 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.591    0.591 3150930661.py:1(getLowUpper_v3)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amax)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amin)
        1    0.000    0.000    0.591    0.591 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2672(_amax_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2677(amax)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2797(_amin_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2802(amin)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:69(_wrapreduction)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:70(<dictcomp>)
       45    0.000    0.000    0.001    0.000 function_base.py:2

In [35]:
cp.run("getLowUpper_v3(12345, 10)")

         894 function calls in 0.555 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.555    0.555 3150930661.py:1(getLowUpper_v3)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amax)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amin)
        1    0.000    0.000    0.555    0.555 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2672(_amax_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2677(amax)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2797(_amin_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2802(amin)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:69(_wrapreduction)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:70(<dictcomp>)
       23    0.000    0.000    0.001    0.000 function_base.py:22

In [36]:
cp.run("getLowUpper_v3(12345, 20)")

         596 function calls in 0.545 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.545    0.545 3150930661.py:1(getLowUpper_v3)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amax)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amin)
        1    0.000    0.000    0.545    0.545 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2672(_amax_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2677(amax)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2797(_amin_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2802(amin)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:69(_wrapreduction)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:70(<dictcomp>)
       12    0.000    0.000    0.000    0.000 function_base.py:22

In [37]:
cp.run("getLowUpper_v3(12345, 50)")

         410 function calls in 0.556 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.556    0.556 3150930661.py:1(getLowUpper_v3)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amax)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amin)
        1    0.000    0.000    0.556    0.556 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2672(_amax_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2677(amax)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2797(_amin_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2802(amin)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:69(_wrapreduction)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:70(<dictcomp>)
        5    0.000    0.000    0.000    0.000 function_base.py:22

In [38]:
cp.run("getLowUpper_v3(12345, 100)")

         404 function calls in 0.662 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.662    0.662 3150930661.py:1(getLowUpper_v3)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amax)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amin)
        1    0.000    0.000    0.662    0.662 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2672(_amax_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2677(amax)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2797(_amin_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2802(amin)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:69(_wrapreduction)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:70(<dictcomp>)
        3    0.000    0.000    0.000    0.000 function_base.py:22

Trying a larger number and comparing it with v1 also shows no real improvement

In [44]:
cp.run("getLowUpper_v3(123456, 20)")

         1748 function calls in 16.554 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.003    0.003   16.553   16.553 2216340445.py:1(getLowUpper_v3)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amax)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amin)
        1    0.000    0.000   16.553   16.553 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2672(_amax_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2677(amax)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2797(_amin_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2802(amin)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:69(_wrapreduction)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:70(<dictcomp>)
       36    0.001    0.000    0.001    0.000 function_base.py:

In [42]:
cp.run("v1.getLowUpper(123456)")

         707 function calls in 16.023 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   16.023   16.023 <string>:1(<module>)
        1    0.001    0.001   16.023   16.023 schmidt_hw2.py:31(getLowUpper)
      703   16.021    0.023   16.021    0.023 schmidt_hw2.py:6(isperfect)
        1    0.000    0.000   16.023   16.023 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




### Conclusion on LowUpper
As shown by the experiments above, vectorizing the function calls made by `getLowUpper` does not seem to lead to time improvements.

## Adaption 2: Vectorize `isperfect`
As `isperfect` loops through all numbers smaller than the input integer `n` there is room for improvement here.

To do so, I create an array until n-2 (Because 4 is the smallest perfect square possible after catching 0 and 1 as exceptions, 2 is included in the array in this case and all larger integers `n` have a square root that is smaller than `n-2`).

Then I square all values in the array and `np.where` returns the index of an element being equal to `n` if it exists. Otherwise, the resulting array is empty and `n` is not perfect. Return either the identified root and True or `n` and False.

In [73]:
def isperfect_v2(n: int ):
    """
        This function is the first helper. It takes an integer n and checks if n has a perfect square root or not.
        If n has a perfect square root, then it returns True and its perfect square root. If not, it returns False and n.

        INPUT: n as an integer.
        OUTPUT: a tuple (bool, int).

        Examples:
        isperfect(0) = (True, 0)
        isperfect(1) = (True, 1)
        isperfect(3) = (False, 3)
        isperfect(16) = (True, 4)
    """
    if n == 0 or n == 1:
        return (True, n)

    ### BEGIN CODE #####
    arr = np.arange(2, n-1, 1)
    index = np.where(arr**2==n)[0]
    if index.size == 0:
        return False, n
    return True, arr[index][0]
    ### END CODE #####

Comparing both implementations shows clear improvement due to vectorization

In [79]:
cp.run("v1.isperfect(12345678)")

         4 function calls in 2.196 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.196    2.196 <string>:1(<module>)
        1    2.196    2.196    2.196    2.196 schmidt_hw2.py:6(isperfect)
        1    0.000    0.000    2.196    2.196 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [80]:
cp.run("isperfect_v2(12345678)")

         8 function calls in 0.070 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.045    0.045    0.069    0.069 1372425994.py:1(isperfect_v2)
        1    0.000    0.000    0.001    0.001 <__array_function__ internals>:177(where)
        1    0.000    0.000    0.070    0.070 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 multiarray.py:341(where)
        1    0.000    0.000    0.070    0.070 {built-in method builtins.exec}
        1    0.022    0.022    0.022    0.022 {built-in method numpy.arange}
        1    0.001    0.001    0.001    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




### Comparison: Vectorize `isperfect` and `getLowUpper` together
As the two functions below show, chaining the vectorization of `isperfect` with that of `getLowUpper` does not improve time complexity. The `getLowUpper_v4` is the same as v1 but uses the vectorized version of `isperfect` while `getLowUpper_v5` uses arrays for finding the next perfect square root. As shown below, v4 is a lot faster.

In [93]:
def getLowUpper_v4(n: int):
    """
    This function is the second helper. It takes an integer n and returns the lower and upper perfect square root to n.
    We will use numpy vectorization to find the lower and upper perfect square roots.

    INPUT: n as an integer.
    OUTPUT: a tuple (minsqrt:int, maxsqrt:int)

    Examples:
    getLowUpper(3) = (1,2)
    getLowUpper(15) = (3,4)
    """
    i = 1
    ### BEGIN CODE ####
    low = isperfect_v2(n-i)
    upper = isperfect_v2(n+i)

    while not low[0]: ## Hint: look at the second while loop.
        i = i+1
        low = isperfect_v2(n-i)

    i = 1
    while not upper[0]:
        i += 1
        upper = isperfect_v2(n+i)

    minsqrt, maxsqrt = low[1], upper[1]
    
    return minsqrt, maxsqrt

In [94]:
def getLowUpper_v5(n: int):
    """
        This function is the second helper. It takes an integer n and returns the lower and upper perfect square root to n.
        We will use two "while" loops here, but we could have used "for" loops or whatever.
        The first that will catch the first perfect square root is less than the square root of n.
        The second one will catch the first square root greater than the square root of n.

        INPUT: n as an integer.
        OUTPUT: a tuple (minsqrt:int, maxsqrt:int)

        Examples:
        getLowUpper(3) = (1,2)
        getLowUpper(15) = (3,4)
    """
    lower_arr, upper_arr = create_ranges(n)
    
    lower_mask, lower_int = np.vectorize(isperfect_v2)(lower_arr)
    minsqrt = lower_int[lower_mask].max()
    
    upper_mask, upper_int = np.vectorize(isperfect_v2)(upper_arr)
    maxsqrt = upper_int[upper_mask].min()

    return minsqrt, maxsqrt

In [99]:
cp.run("getLowUpper_v4(123456)")

         3519 function calls in 0.075 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.075    0.075 1360873061.py:1(getLowUpper_v4)
      703    0.036    0.000    0.073    0.000 1372425994.py:1(isperfect_v2)
      703    0.003    0.000    0.009    0.000 <__array_function__ internals>:177(where)
        1    0.000    0.000    0.075    0.075 <string>:1(<module>)
      703    0.001    0.000    0.001    0.000 multiarray.py:341(where)
        1    0.000    0.000    0.075    0.075 {built-in method builtins.exec}
      703    0.028    0.000    0.028    0.000 {built-in method numpy.arange}
      703    0.004    0.000    0.004    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [100]:
cp.run("getLowUpper_v5(123456)")

         50691 function calls in 1.021 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10125    0.499    0.000    1.002    0.000 1372425994.py:1(isperfect_v2)
        1    0.000    0.000    1.021    1.021 1900466169.py:1(getLowUpper_v5)
        1    0.000    0.000    0.000    0.000 3018333493.py:1(create_ranges)
    10125    0.043    0.000    0.118    0.000 <__array_function__ internals>:177(where)
        1    0.000    0.000    1.021    1.021 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _methods.py:38(_amax)
        1    0.000    0.000    0.000    0.000 _methods.py:42(_amin)
        2    0.000    0.000    0.000    0.000 function_base.py:2268(__init__)
        2    0.000    0.000    1.020    0.510 function_base.py:2300(__call__)
        2    0.000    0.000    0.001    0.001 function_base.py:2330(_get_ufunc_and_otypes)
        2    0.000    0.000    0.000    0.000 function_base.py:2360(<listcomp>)
 

In [101]:
cp.run("v1.getLowUpper(123456)")

         707 function calls in 15.384 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   15.384   15.384 <string>:1(<module>)
        1    0.001    0.001   15.384   15.384 schmidt_hw2.py:31(getLowUpper)
      703   15.383    0.022   15.383    0.022 schmidt_hw2.py:6(isperfect)
        1    0.000    0.000   15.384   15.384 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




## Adaption 3: Precalculating `sqrt(n)` in `mysqrt`
As a small adaption, I want to also evaluate if there is benefit in calculating `np.sqrt(n)` once and reusing this in each loop of `mysqrt`. Apart from this change, v1 and v2 are identical

In [108]:
def mysqrt_v1(n: int, error_threshold=0.000000001) -> float:
    """
        This function is the main function. It takes an integer n and returns the square root of n.
        We will use here the two helper functions we wrote previously.


        INPUT: n as an integer.
        OUTPUT: a float rst

        Examples:
        mysqrt(3) = 1.7320508076809347
        mysqrt(15) = 3.8729833462275565
    """

    ### BEGIN CODE ###
    if n == 0 or n == 1: ## Hint: remember to always start by basic case solution. for the square root problem, we have 0 and 1
        return n
    ### END CODE ###



    ### BEGIN CODE ###
    checkup = isperfect_v2(n) # Hint: use the one of the helpers you already coded.
    if checkup[0] : # How to access an element of the tuple?
        return checkup[1] #Choose the right index...
    ### END CODE ###

    iteration = 0 # The variable is used to count the number of times we repeat the instructions in the while loop

    ### BEGING CODE ###
    minsqrt, maxsqrt = getLowUpper_v4(n) #Hint: use the second helper function.

    rst =  (minsqrt + maxsqrt) / 2
    
    while abs(np.sqrt(n)-rst) >= error_threshold:
            if rst**2 < n : # Hint: have a look at the first function.
                    minsqrt = rst
            else :
                    maxsqrt = rst
            rst = (minsqrt + maxsqrt) / 2
            iteration +=1
    ### END CODE ####

    return rst

In [109]:
def mysqrt_v2(n: int, error_threshold=0.000000001) -> float:
    """
        This function is the main function. It takes an integer n and returns the square root of n.
        We will use here the two helper functions we wrote previously.


        INPUT: n as an integer.
        OUTPUT: a float rst

        Examples:
        mysqrt(3) = 1.7320508076809347
        mysqrt(15) = 3.8729833462275565
    """

    ### BEGIN CODE ###
    if n == 0 or n == 1: ## Hint: remember to always start by basic case solution. for the square root problem, we have 0 and 1
        return n
    ### END CODE ###



    ### BEGIN CODE ###
    checkup = isperfect_v2(n) # Hint: use the one of the helpers you already coded.
    if checkup[0] : # How to access an element of the tuple?
        return checkup[1] #Choose the right index...
    ### END CODE ###

    iteration = 0 # The variable is used to count the number of times we repeat the instructions in the while loop

    ### BEGING CODE ###
    minsqrt, maxsqrt = getLowUpper_v4(n) #Hint: use the second helper function.

    rst =  (minsqrt + maxsqrt) / 2
    np_sqrt = np.sqrt(n)
    
    while abs(np_sqrt-rst) >= error_threshold:
            if rst**2 < n : # Hint: have a look at the first function.
                    minsqrt = rst
            else :
                    maxsqrt = rst
            rst = (minsqrt + maxsqrt) / 2
            iteration +=1
    ### END CODE ####

    return rst

In [119]:
cp.run("mysqrt_v1(1234567, 0.000000000001)")

         11162 function calls in 5.321 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.009    0.009    5.315    5.315 1360873061.py:1(getLowUpper_v4)
     2224    3.481    0.002    5.311    0.002 1372425994.py:1(isperfect_v2)
        1    0.000    0.000    5.321    5.321 524931815.py:1(mysqrt_v1)
     2224    0.015    0.000    0.238    0.000 <__array_function__ internals>:177(where)
        1    0.000    0.000    5.321    5.321 <string>:1(<module>)
     2224    0.004    0.000    0.004    0.000 multiarray.py:341(where)
       37    0.000    0.000    0.000    0.000 {built-in method builtins.abs}
        1    0.000    0.000    5.321    5.321 {built-in method builtins.exec}
     2224    1.592    0.001    1.592    0.001 {built-in method numpy.arange}
     2224    0.218    0.000    0.218    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.000    0.000 {m

In [120]:
cp.run("mysqrt_v2(1234567, 0.000000000001)")

         11162 function calls in 4.970 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.009    0.009    4.965    4.965 1360873061.py:1(getLowUpper_v4)
     2224    3.244    0.001    4.961    0.002 1372425994.py:1(isperfect_v2)
        1    0.000    0.000    4.970    4.970 2809391626.py:1(mysqrt_v2)
     2224    0.015    0.000    0.233    0.000 <__array_function__ internals>:177(where)
        1    0.000    0.000    4.970    4.970 <string>:1(<module>)
     2224    0.004    0.000    0.004    0.000 multiarray.py:341(where)
       37    0.000    0.000    0.000    0.000 {built-in method builtins.abs}
        1    0.000    0.000    4.970    4.970 {built-in method builtins.exec}
     2224    1.483    0.001    1.483    0.001 {built-in method numpy.arange}
     2224    0.214    0.000    0.214    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.000    0.000 {

### Conclusion

While there were some differences between v2, these were not consistent between different tries (even for the same number).

# Overall conclusion
As shown by adaptions 1 and 2, vectorizing `isperfect` leads to very noticeable improvement in time complexity while trying to vectorize `getLowUpper` did not prove successful.

A comparison of the different implementations can be found in the notebook evaluation_hw3.